In [1]:
pwd #working directory

'/home/wsuser/work'

In [2]:
!pip install keras==2.2.4

  Using cached Keras-2.2.4-py2.py3-none-any.whl (312 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.


In [3]:
!pip install tensorflow

  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1)

In [7]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='OkcLD59yDhebUrg9qRuw15Ct5aSoTDv4tkWsrevCO2Q6',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'fruitmodel-donotdelete-pr-9jnnkerxijcswa'
object_key = 'fruit-dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [8]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()), "r")
file_paths = unzip.namelist() 
for path in file_paths:
    unzip.extract(path)

In [9]:
import os
filenames = os.listdir('/home/wsuser/work/fruit-dataset')

In [10]:
x_train=train_datagen.flow_from_directory("/home/wsuser/work/fruit-dataset/train",target_size=(128,128),class_mode='categorical',batch_size=32)

Found 5384 images belonging to 6 classes.


In [11]:
x_test=test_datagen.flow_from_directory(r"/home/wsuser/work/fruit-dataset/test",target_size=(128,128),
                                        class_mode='categorical',batch_size=32)

Found 1686 images belonging to 6 classes.


In [12]:
x_train.class_indices

{'Apple___Black_rot': 0,
 'Apple___healthy': 1,
 'Corn_(maize)___Northern_Leaf_Blight': 2,
 'Corn_(maize)___healthy': 3,
 'Peach___Bacterial_spot': 4,
 'Peach___healthy': 5}

In [13]:
model=Sequential() #load the model

In [14]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu')) #add convolutional layer

In [15]:
model.add(MaxPooling2D(pool_size=(2,2))) #add max pooling layer

In [16]:
model.add(Flatten()) #add flatten layer

In [17]:
model.add(Dense(units=128,kernel_initializer='uniform',activation='relu')) #add dense hidden layer

In [18]:
model.add(Dense(units=6,kernel_initializer='uniform',activation='softmax')) #add the output layer

In [19]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=["accuracy"]) #compile the model

In [20]:
model.fit(x_train,steps_per_epoch=168,epochs=5,validation_data=x_test,validation_steps=52)

Epoch 1/5
168/168 [==============================] - 81s 481ms/step - loss: 0.6966 - accuracy: 0.7754 - val_loss: 36.6627 - val_accuracy: 0.8954
Epoch 2/5
168/168 [==============================] - 81s 479ms/step - loss: 0.2758 - accuracy: 0.9062 - val_loss: 202.9319 - val_accuracy: 0.7218
Epoch 3/5
168/168 [==============================] - 83s 492ms/step - loss: 0.2287 - accuracy: 0.9225 - val_loss: 147.2584 - val_accuracy: 0.7626
Epoch 4/5
168/168 [==============================] - 80s 473ms/step - loss: 0.1915 - accuracy: 0.9335 - val_loss: 476.9718 - val_accuracy: 0.6250
Epoch 5/5
168/168 [==============================] - 80s 472ms/step - loss: 0.1656 - accuracy: 0.9423 - val_loss: 458.1934 - val_accuracy: 0.5306


In [23]:
model.save('fruitdata.h5') #store model to use it locally

In [67]:
!tar -zcvf Train-model_new.tgz fruitdata.h5 #convert to tgz file named as image-classification-model_new

fruitdata.h5


In [68]:
#store model in repository

In [69]:
!pip install watson-machine-learning-client --upgrade

In [70]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"Hdf86j0RNd5M-Y_NNeDjz9U9Xp2UC_cnj2Ij3QBADfeD" 
                  }

client = APIClient(wml_credentials)

In [71]:
client = APIClient(wml_credentials)

In [72]:
def guid_from_space_name(client, space_name): 
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [73]:
space_uid = guid_from_space_name(client, 'image_classification_fruits') 
print("Space UID = " + space_uid)

Space UID = 7e5d8a6a-86b5-4ca2-8894-512b8f1106d4


In [74]:
client.set.default_space(space_uid)

'SUCCESS'

In [75]:
client.software_specifications.list(limit=25)

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [76]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")

In [77]:
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [78]:
import tensorflow
print(tensorflow.__version__)

2.7.2


In [79]:
ls

'Dataset Plant Disease'/   image-classification-model_new.tgz
 fruitdata.h5              Train-model_new.tgz
 fruit-dataset/


In [80]:
model_details = client.repository.store_model(model= 'Train-model_new.tgz',
    meta_props={
        client.repository.ModelMetaNames.NAME:"CNN",
        client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
    )

In [81]:
model_id = client.repository.get_model_id(model_details)

In [82]:
model_id

'1a9d5948-8db6-4714-bc5c-fcf9dde56924'

In [83]:
client.repository.download(model_id,'final_fruit_model.tar.gz')

Successfully saved model content to file: 'final_fruit_model.tar.gz'


'/home/wsuser/work/final_fruit_model.tar.gz'

In [ ]:
#test the model

In [84]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [86]:
model=load_model('fruitdata.h5')

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='OkcLD59yDhebUrg9qRuw15Ct5aSoTDv4tkWsrevCO2Q6',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'fruitmodel-donotdelete-pr-9jnnkerxijcswa'
object_key = 'peach_bacteria.JPG'

streaming_body_4 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

img= image.load_img(streaming_body_4,target_size=(64,64))

In [ ]:
x=image.img_to_array(img)

In [ ]:
y=np.argmax(model.predict(x),axis=1)

In [ ]:
index[y[0]]

In [ ]:
index=['Apple___Black_rot','Apple___healthy','Corn_(maize)___Northern_Leaf_Blight','Corn_(maize)___healthy','Peach___Bacterial_spot','Peach___healthy']

In [ ]:
!pip install numpy
!pip install flask
!pip install os
!pip install werkzeug

In [ ]:
import requests
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import tensorflow as tf
from flask import Flask, request, render_template, redirect, url_for
import os
from werkzeug.utils import secure_filename

In [ ]:
app = Flask(__name__)

#load both the vegetable and fruit models
model = load_model("IBM-vegetable.h5")
model1=load_model("IBM-fruit.h5")

#home page
@app.route('/')
def home():
    return render_template('home.html')

#prediction page
@app.route('/prediction')
def prediction():
    return render_template('predict.html')

@app.route('/predict',methods=['POST'])		
def predict():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['image']

        # Save the file to ./uploads
        basepath = os.path.dirname(__file__)
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        img = image.load_img(file_path, target_size=(128, 128))
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        plant=request.form['plant']
        print(plant)
        if(plant=="vegetable"):
            preds = model.predict(x)
            preds=np.argmax(preds)
            print(preds)
            df=pd.read_excel('precautions - veg.xlsx')
            print(df.iloc[preds]['caution'])
        else:
            preds = model1.predict(x)
            preds=np.argmax(preds)                
            df=pd.read_excel('precautions - fruits.xlsx')
            print(df.iloc[preds]['caution'])
            
        
        return df.iloc[preds]['caution']
        
if __name__ == "__main__":
    app.run(debug=False)